In [22]:
import pandas as pd
print(" pandas version:", pd.__version__)


 pandas version: 2.3.1


In [8]:
import pandas as pd
import os

# Since notebook and CSVs are in the same folder
data_path = "./"

# Load all files
user = pd.read_csv(os.path.join(data_path, "user.csv"))
user_registration = pd.read_csv(os.path.join(data_path, "user_registration.csv"))
user_plan = pd.read_csv(os.path.join(data_path, "user_plan.csv"))
user_payment_detail = pd.read_csv(os.path.join(data_path, "user_payment_detail.csv"))
user_play_session = pd.read_csv(os.path.join(data_path, "user_play_session.csv"))
channel_code = pd.read_csv(os.path.join(data_path, "channel_code.csv"))
status_code = pd.read_csv(os.path.join(data_path, "status_code.csv"))
plan = pd.read_csv(os.path.join(data_path, "plan.csv"))


# Preview to confirm
print(user.head(2))
print(user_play_session.head(2))


   user_id       ip_address social_media_handle                   email
0        1  192.168.192.126            lherring                     NaN
1        2    172.18.54.245          jrodriguez  jrodriguez@example.com
   play_session_id  user_id                 start_datetime  \
0                0      309  2024-12-11T12:26:22.670-06:00   
1                1      179  2024-02-13T15:51:16.919-06:00   

                    end_datetime channel_code status_code  total_score  
0  2024-12-11T17:01:22.670-06:00      BROWSER   COMPLETED         1081  
1  2024-02-13T22:17:16.919-06:00       MOBILE     ABORTED         4271  


In [24]:
import os
os.listdir()


['.ipynb_checkpoints',
 'channel_code.csv',
 'dice_game_etl.ipynb',
 'output',
 'plan.csv',
 'status_code.csv',
 'user.csv',
 'user_payment_detail.csv',
 'user_plan.csv',
 'user_play_session.csv',
 'user_registration.csv']

In [25]:
# STEP 1: Import libraries
import pandas as pd
import os

# STEP 2: Load data
data_path = "."

user = pd.read_csv(os.path.join(data_path, "user.csv"))
user_registration = pd.read_csv(os.path.join(data_path, "user_registration.csv"))
user_plan = pd.read_csv(os.path.join(data_path, "user_plan.csv"))
user_payment_detail = pd.read_csv(os.path.join(data_path, "user_payment_detail.csv"))
user_play_session = pd.read_csv(os.path.join(data_path, "user_play_session.csv"))
channel_code = pd.read_csv(os.path.join(data_path, "channel_code.csv"))
status_code = pd.read_csv(os.path.join(data_path, "status_code.csv"))
plan = pd.read_csv(os.path.join(data_path, "plan.csv"))

# STEP 3: Create dimension tables
dim_user = user.merge(user_registration, on="user_id", how="left")
dim_user = dim_user[["user_id", "username", "first_name", "last_name", "email_y", "ip_address", "social_media_handle"]]
dim_user.rename(columns={"email_y": "email"}, inplace=True)

dim_plan = plan.copy()
dim_payment = user_payment_detail.copy()

# STEP 4: Create fact tables
fact_play = user_play_session \
    .merge(channel_code, left_on="channel_code", right_on="play_session_channel_code", how="left") \
    .merge(status_code, left_on="status_code", right_on="play_session_status_code", how="left") \
    .merge(user, on="user_id", how="left")

fact_user_plan = user_plan \
    .merge(plan, on="plan_id", how="left") \
    .merge(user_payment_detail, on="payment_detail_id", how="left")

# STEP 5: Data Quality Checks
print(" Running Data Quality Checks...")
assert dim_user['user_id'].is_unique, " user_id not unique in dim_user"

if fact_play.isnull().sum().sum() > 0:
    print("Missing values in fact_play — filling with 'Unknown'")
    fact_play.fillna("Unknown", inplace=True)

if fact_user_plan.isnull().sum().sum() > 0:
    print("Missing values in fact_user_plan — filling with 'Unknown'")
    fact_user_plan.fillna("Unknown", inplace=True)

# STEP 6: Key Business Insights
print("\n Insight 1: Online vs Mobile sessions")
print(fact_play['english_description_x'].value_counts())

print("\n Insight 2: Onetime vs Subscription Plans")
print(fact_user_plan['payment_frequency_code'].value_counts())

print("\n Insight 3: Gross Revenue")
total_revenue = fact_user_plan["cost_amount"].sum()
print(f"Total Revenue: ${total_revenue:,.2f}")

# STEP 7: Export to output
output_path = "./output"
os.makedirs(output_path, exist_ok=True)

dim_user.to_csv(f"{output_path}/dim_user.csv", index=False)
dim_plan.to_csv(f"{output_path}/dim_plan.csv", index=False)
dim_payment.to_csv(f"{output_path}/dim_payment.csv", index=False)
fact_play.to_csv(f"{output_path}/fact_play.csv", index=False)
fact_user_plan.to_csv(f"{output_path}/fact_user_plan.csv", index=False)

print("\n All tables saved to ./output folder.")




 Running Data Quality Checks...
Missing values in fact_play — filling with 'Unknown'

 Insight 1: Online vs Mobile sessions
english_description_x
Browser               941
Mobile application    931
Name: count, dtype: int64

 Insight 2: Onetime vs Subscription Plans
payment_frequency_code
ONETIME     75
MONTHLY     74
ANNUALLY    66
Name: count, dtype: int64

 Insight 3: Gross Revenue
Total Revenue: $1,930.85

 All tables saved to ./output folder.
